In [60]:
import os
os.chdir('/content')
# 기존 경로에 존재하는 프로젝트는 지우고 다시 다운로드 받고 시작
!rm -rf /content/GOLD
!git clone https://github.com/2CHAN0/GOLD.git

# 프로젝트로 이동 후 라이브러리 설치
os.chdir('GOLD')
!pip install -q -r requirements.txt

Cloning into 'GOLD'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 259 (delta 139), reused 194 (delta 74), pack-reused 0 (from 0)
Receiving objects: 100% (259/259), 97.37 KiB | 3.61 MiB/s, done.
Resolving deltas: 100% (139/139), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# 전역변수
GOOGLE_DRIVE_BASE_PATH='/content/drive/MyDrive/GOLD/gold-style-on-policy-only'
TEACHER_MODEL='Qwen/Qwen2.5-3B-Instruct'
STUDENT_MODEL='Qwen/Qwen2.5-1.5B-Instruct'
CHECKPOINT_STEP=114
NGROK_AUTH_TOKEN=''

# 환경변수
os.environ["NGROK_AUTH_TOKEN"] = NGROK_AUTH_TOKEN

In [ ]:
# Training ODP
!accelerate launch scripts/train_gold_style.py \
  --output-dir {GOOGLE_DRIVE_BASE_PATH} \
  --max-steps {CHECKPOINT_STEP} \
  --teacher {TEACHER_MODEL} \
  --student {STUDENT_MODEL} \
  --prompt-source dynamic \
  --per-device-train-batch-size 1 \
  --gradient-accumulation-steps 2 \
  --learning-rate 1e-4 \
  --lmbda 1.0 \
  --beta 0.3 \
  --dtype bfloat16 \
  --max-completion-length 128 \
  --trust-remote-code \
  --no-use-student-system-prompt \
  --fix-mistral-regex

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/content/GOLD/scripts/train_gold_style.py:27: UserWarning: You are importing from 'trl.experimental'. APIs here are unstable and may change or be removed without notice. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  from trl.experimental.gold import GOLDConfig, GOLDTrainer
2025-11-30 11:52:13.315713: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-30 1

In [48]:
# Inference Test
!python scripts/eval_style_responses.py \
  --model-path {GOOGLE_DRIVE_BASE_PATH}/checkpoint-{CHECKPOINT_STEP} \
  --base-model {STUDENT_MODEL} \
  --prompts-file data/style_toggles.jsonl \
  --max-new-tokens 128

2025-11-30 11:11:36.885247: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-30 11:11:36.906980: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764501096.928618   54072 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764501096.934601   54072 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764501096.950555   54072 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [59]:
# Serving with Ngrok
!python scripts/serve_style_ngrok.py \
  --model-path {STUDENT_MODEL} \
  --base-model {STUDENT_MODEL} \
  --secondary-model-path {GOOGLE_DRIVE_BASE_PATH}/checkpoint-{CHECKPOINT_STEP} \
  --secondary-base-model {STUDENT_MODEL}


2025-11-30 11:43:42.559125: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-30 11:43:42.578549: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764503022.601879   62622 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764503022.609278   62622 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764503022.625122   62622 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 